<a href="https://colab.research.google.com/github/akatwinkle/bab_bab/blob/main/%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'
url


'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'

In [16]:
import requests

url = 'https://data.seoul.go.kr/dataList/OA-15361/S/1/datasetView.do'
api_key = '4a48554755796f6f36375856727974'

params = {
    'KEY' : api_key,
    'TYPE' : 'json',
    'SERVICE' : 'energyUseDataSummaryInfo',
    'START_INDEX' : '1',
    'END_INDEX' : '100'
}

def get_year_month():
    year_month = []
    for year in range(2015, 2024+1):
        for month in range(1,12+1):
            year_month.append(f'{year}{month:02d}')
    return year_month

resp = []

for year_month in get_year_month():

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()

        data = resp.json()

        print("API 호출 성공", data)
        print(data)

    except requests.exceptions.RequestException as e:
        print(f"API 호출 실패: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 실패: {e}")

API 호출 실패: Expecting value: line 23 column 1 (char 57)
API 호출 실패: Expecting value: line 23 column 1 (char 57)
API 호출 실패: Expecting value: line 23 column 1 (char 57)
API 호출 실패: Expecting value: line 23 column 1 (char 57)
API 호출 실패: Expecting value: line 23 column 1 (char 57)


NameError: name 'json' is not defined